In [22]:
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from pycbc.waveform import get_td_waveform
from pycbc.noise import noise_from_psd
from pycbc.psd import aLIGOZeroDetHighPower
from pycbc.types import TimeSeries
from pycbc.filter import sigma
from scipy.signal import spectrogram

# --- Parameters ---
num_samples = 1000
sample_rate = 4096
duration = 4  # seconds
samples = sample_rate * duration
save_dir = "results"
np.random.seed(42)

# --- Output folder ---
os.makedirs(save_dir, exist_ok=True)

# --- Label setup ---
yes_signal = num_samples // 2
no_signal = num_samples - yes_signal
labels = [1] * yes_signal + [0] * no_signal
np.random.shuffle(labels)

# --- PSD for both signal injection and whitening ---
psd = aLIGOZeroDetHighPower(
    length=samples // 2 + 1,
    delta_f=1.0 / (samples * (1.0 / sample_rate)),
    low_freq_cutoff=20
)

x = []
y = []
meta = []

# --- Generate Samples ---
for i in tqdm(range(num_samples)):
    target_snr = np.random.uniform(8, 20)

    # --- Noise ---
    noise_ts = noise_from_psd(
        length=samples,
        delta_t=1/sample_rate,
        psd=psd,
        seed=i
    )

    signal_ts = None
    mass1 = mass2 = snr = None

    if labels[i]:  # signal
        mass1 = np.random.uniform(5, 80)
        mass2 = np.random.uniform(5, 80)

        hp, _ = get_td_waveform(
            approximant="IMRPhenomXPHM",
            mass1=mass1,
            mass2=mass2,
            spin1z=np.random.uniform(-0.9, 0.9),
            spin2z=np.random.uniform(-0.9, 0.9),
            distance=np.random.uniform(300, 800),
            inclination=np.random.uniform(0, np.pi),
            delta_t=1.0 / sample_rate,
            f_lower=30
        )

        signal = hp.numpy()

        # --- Center the waveform ---
        if len(signal) < samples:
            pad_total = samples - len(signal)
            left_pad = pad_total // 2
            right_pad = pad_total - left_pad
            signal = np.pad(signal, (left_pad, right_pad))
        else:
            signal = signal[:samples]

        signal_ts = TimeSeries(signal, delta_t=1.0/sample_rate)

        # --- Scale to target SNR ---
        optimal_snr = sigma(signal_ts, psd=psd, low_frequency_cutoff=20)
        scaling_factor = target_snr / optimal_snr
        snr = target_snr
        scaled_signal = signal_ts * scaling_factor
        injected_ts = noise_ts + scaled_signal

    else:
        injected_ts = noise_ts

    # --- Whitening ---
    whitened = injected_ts.whiten(4, 2)

    # --- Spectrogram ---
    f, t, Sxx = spectrogram(
        whitened,
        fs=sample_rate,
        nperseg=1024,
        noverlap=512
    )

    # --- Log scale & normalize ---
    Sxx_log = 10 * np.log10(Sxx + 1e-10)
    std = np.std(Sxx_log)
    if std == 0:
        std = 1e-6
    Sxx_log = (Sxx_log - np.mean(Sxx_log)) / std

    # --- Pad or truncate to fixed time length ---
    if Sxx_log.shape[1] < 31:
        pad = 31 - Sxx_log.shape[1]
        Sxx_log = np.pad(Sxx_log, ((0, 0), (0, pad)), mode='constant')
    else:
        Sxx_log = Sxx_log[:, :31]

    # --- Save ---
    x.append(Sxx_log)
    y.append(labels[i])
    meta.append({
        "index": i,
        "label": labels[i],
        "snr": float(snr) if snr is not None else None,
        "mass1": float(mass1) if mass1 is not None else None,
        "mass2": float(mass2) if mass2 is not None else None,
    })

# --- Final arrays ---
x = np.array(x)[..., np.newaxis]  # shape: (N, F, T, 1)
y = np.array(y)

np.save(os.path.join(save_dir, "x.npy"), x)
np.save(os.path.join(save_dir, "y.npy"), y)

print("✅ Dataset saved:")
print("  X shape:", x.shape)
print("  y shape:", y.shape)


100%|██████████| 1000/1000 [07:50<00:00,  2.13it/s]


✅ Dataset saved:
  X shape: (1000, 513, 31, 1)
  y shape: (1000,)
